In [6]:
#create db tables and load csv files
import csv
import codecs
import sqlite3
import pprint

table_list = ['nodes', 'nodes_tags', 'ways', 'ways_tags', 'ways_nodes']

con = sqlite3.connect("Houston_Tx.db")
cur = con.cursor()

#drop tables if they exists so we do not insert repeat data
for tablename in table_list:
    stmt = "DROP TABLE IF EXISTS " + tablename
    cur.execute(stmt)
    con.commit()

# create nodes table
cur.execute("CREATE TABLE IF NOT EXISTS nodes (b,id, lat, lon, user, uid, version, changeset, timestamp);")

# load table
with codecs.open('nodes.csv', encoding='utf-8-sig') as fin:
    dr = csv.DictReader(fin)
    pprint.pprint(dr.fieldnames)
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) \
                VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)


# create nodes_tags table
cur.execute("CREATE TABLE IF NOT EXISTS nodes_tags (id, key, value, type);")

# load table
with codecs.open('nodes_tags.csv', encoding='utf-8-sig') as fin:
    dr = csv.DictReader(fin)
    pprint.pprint(dr.fieldnames)
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)

# create ways table
cur.execute("CREATE TABLE IF NOT EXISTS ways (id, user, uid, version, changeset, timestamp);")

# load table
with codecs.open('ways.csv', encoding='utf-8-sig') as fin:
    dr = csv.DictReader(fin)
    pprint.pprint(dr.fieldnames)
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)


# create ways_nodes table
cur.execute("CREATE TABLE IF NOT EXISTS ways_nodes (id, node_id, position);")

# load table
with codecs.open('ways_nodes.csv', encoding='utf-8-sig') as fin:
    dr = csv.DictReader(fin)
    pprint.pprint(dr.fieldnames)
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?, ?, ?);", to_db)

# create ways_tags table
cur.execute("CREATE TABLE IF NOT EXISTS ways_tags (id, key, value, type);")

# load table
with codecs.open('ways_tags.csv', encoding='utf-8-sig') as fin:
    dr = csv.DictReader(fin)
    pprint.pprint(dr.fieldnames)
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)

con.commit()
#Count of nodes, nodes tags, ways, ways tags, and ways nodes.
for tablename in table_list:
    stmt = 'SELECT COUNT(*) FROM ' + tablename
    cur.execute(stmt)
    rows = cur.fetchall()
    print("--------------------------------------------------------------")
    print(tablename + " Row Count:", rows)


con.close()

['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
['id', 'key', 'value', 'type']
['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
['id', 'node_id', 'position']
['id', 'key', 'value', 'type']
--------------------------------------------------------------
nodes Row Count: [(1167464,)]
--------------------------------------------------------------
nodes_tags Row Count: [(57019,)]
--------------------------------------------------------------
ways Row Count: [(158927,)]
--------------------------------------------------------------
ways_tags Row Count: [(500445,)]
--------------------------------------------------------------
ways_nodes Row Count: [(1402217,)]


In [7]:
# number of nodes 

cur.execute('SELECT COUNT(*) FROM nodes')
all_rows = cur.fetchall()
print(all_rows)
# number of ways
cur.execute('SELECT COUNT(*) FROM ways')
all_rows = cur.fetchall()
print(all_rows)

[(1167464,)]
[(158927,)]


In [8]:
#Top 3 users
QUERY = '''
SELECT DISTINCT nodes.user, COUNT(*)
FROM nodes
GROUP BY nodes.uid
ORDER BY COUNT(*) DESC
LIMIT 3;
'''

cur.execute(QUERY)
all_rows = cur.fetchall()
print(all_rows)

[('afdreher', 91474), ('scottyc', 81680), ('skquinn', 78720)]


In [17]:
# % of nodes users
QUERY = '''
SELECT DISTINCT nodes.user, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM nodes)
FROM nodes
GROUP BY nodes.uid
ORDER BY (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM nodes)) DESC
LIMIT 10;
'''

cur.execute(QUERY)
all_rows = cur.fetchall()
print(all_rows)

[('afdreher', 7.835273721502333), ('scottyc', 6.996361343904394), ('skquinn', 6.742820335359378), ('woodpeck_fixbot', 5.737821466015226), ('TexasNHD', 3.3022859805527194), ('cammace', 2.985531031363708), ('m_with_z', 2.2261071861744774), ('clay_c', 2.177026443641945), ('antiviro', 1.880229283301241), ('Memoire', 1.7700759937779666)]


In [9]:
#Street names
QUERY = '''
SELECT ways_tags.value, COUNT(*)
FROM ways_tags
WHERE ways_tags.key = 'name'
AND ways_tags.type = 'regular'
GROUP BY ways_tags.value
ORDER BY COUNT(*) DESC
LIMIT 10;
'''

cur.execute(QUERY)
all_rows = cur.fetchall()
print(all_rows)

[('West Little York Road', 67), ('West Road', 65), ('Mason Road', 65), ('Cypresswood Drive', 64), ('North Eldridge Parkway', 63), ('Westheimer Road', 62), ('Eastex Freeway Frontage Road', 62), ('Eastex Freeway', 61), ('Barker Cypress Road', 61), ('T C Jester Boulevard', 58)]


In [10]:
#Average of ways
QUERY = '''
SELECT AVG(Count)
FROM
    (SELECT COUNT(*) as Count
    FROM ways
    JOIN ways_nodes
    ON ways.id = ways_nodes.id
    GROUP BY ways.id);
'''

cur.execute(QUERY)
all_rows = cur.fetchall()
print(all_rows)

[(8.823025665871752,)]


In [11]:
#Amenities
QUERY = '''
SELECT value, COUNT(*) as Count
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
ORDER BY Count DESC
LIMIT 10;
'''

cur.execute(QUERY)
all_rows = cur.fetchall()
print(all_rows)

[('parking_space', 404), ('place_of_worship', 372), ('fountain', 303), ('restaurant', 243), ('fast_food', 177), ('bench', 157), ('parking', 87), ('waste_basket', 71), ('fuel', 71), ('school', 59)]


In [12]:
#Religion
QUERY = '''
SELECT nodes_tags.value, COUNT(*) as Count
FROM nodes_tags 
JOIN
    (SELECT DISTINCT(id)
    FROM nodes_tags
    WHERE value='place_of_worship') as Sub
ON nodes_tags.id=Sub.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY Count DESC;
'''

cur.execute(QUERY)
all_rows = cur.fetchall()
print(all_rows)

[('christian', 359), ('muslim', 6), ('buddhist', 5), ('jewish', 1), ('hindu', 1)]


In [13]:
#Cities
QUERY = '''
SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) tags
WHERE tags.key = 'city'
GROUP BY tags.value
ORDER BY count DESC;
'''

cur.execute(QUERY)
all_rows = cur.fetchall()
print(all_rows)



[('Houston', 1841), ('Katy', 151), ('Tomball', 73), ('Kingwood', 61), ('Baytown', 56), ('Richmond', 52), ('Bellaire', 50), ('Pearland', 45), ('Spring', 42), ('Sugar Land', 41), ('Humble', 39), ('Klein', 38), ('Dickinson', 32), ('Manvel', 28), ('Fulshear', 27), ('New Caney', 26), ('Cypress', 24), ('Missouri City', 22), ('The Woodlands', 19), ('Magnolia', 18), ('Webster', 17), ('Seabrook', 16), ('League City', 16), ('Porter', 13), ('Pasadena', 9), ('Tomball, Tx', 6), ('Friendswood', 6), ('Stafford', 5), ('Rosenberg', 5), ('Shenandoah', 4), ('Conroe', 4), ('West University', 3), ('Liberty', 3), ('La Porte', 3), ('Hockley', 3), ('Crosby', 3), ('Waller', 2), ('Sugarland', 2), ('Sheldon', 2), ('Prairie View', 2), ('Nassau Bay', 2), ('Mont Belvieu', 2), ('Hempstead', 2), ('Deer Park', 2), ('Crystal Beach', 2), ('Clear Lake Shores', 2), ('CONROE', 2), ('Bellville', 2), ('Bacliff', 2), ('Alvin', 2), ('houston', 1), ('West University Place', 1), ('Wallis', 1), ('Texas City', 1), ('Street', 1), (

In [14]:
#Phone numbers
QUERY = '''
SELECT tags.value
FROM (SELECT * FROM nodes_tags UNION 
      SELECT * FROM ways_tags) tags
WHERE tags.key = 'phone' and tags.value NOT LIKE '+49 ___ %';
'''

cur.execute(QUERY)
all_rows = cur.fetchall()
print(all_rows)


[('+1-281-237-7300',), ('+1-281-340-0900',), ('+1-281-237-8000',), ('+1 936 266 2000',), ('+1-936-273-7724',), ('+1-281-292-2080',), ('(713) 467-9343',), ('+1 713 686 9494',), ('+1-713-337-7321',), ('+1 713 790 1234',), ('+1-713-647-5900',), ('+1-281-504-0273',), ('+1-713-782-0982',), ('+1-281-980-1800',), ('+1-713-667-6531',), ('+1-713-939-1826',), ('+1-281-419-7878',), ('+1-713-939-0800',), ('+1 713 228 9472',), ('+1 281 339 2895',), ('+1-281-478-6563',), ('+1-281-436-0987',), ('+1-281-812-4357',), ('+1-281-338-1762',), ('+1-281-446-5300',), ('+1-281-364-7744',), ('+1-281-354-3400',), ('+1-281-275-8400',), ('+1-713-526-4034',), ('+1-281-275-2825',), ('+1 713 272 2600',), ('+1 713 272 2600',), ('+1-281-240-2000',), ('+1-713-218-1800',), ('+1 713 528 6800',), ('+1-281-855-1604',), ('+1-800-440-9031',), ('+1-281-858-5774',), ('+1 713 501 0500',), ('+1 713 413 3400',), ('+1-713-758-7200',), ('+1-281-444-5768',), ('+1-281-421-3800',), ('+1-800-435-9165',), ('+1-713-578-6100',), ('+1 713 8

In [15]:
#Zipcodes
QUERY = '''
SELECT tags.value
FROM (SELECT * FROM nodes_tags) tags
WHERE tags.key = 'postcode';
'''

cur.execute(QUERY)
all_rows = cur.fetchall()
print(all_rows)



[('77573',), ('77004',), ('77379',), ('77044',), ('77016',), ('77471',), ('77471',), ('77520',), ('77357',), ('77004',), ('77429',), ('77084',), ('77584',), ('77373',), ('77355',), ('77002',), ('77034',), ('77077',), ('77002',), ('77518',), ('77386',), ('77571',), ('77027',), ('77027',), ('77005',), ('77027',), ('77571',), ('77536',), ('77041',), ('77072',), ('77030',), ('77054',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77098',), ('77535',), ('77535',), ('77535',), ('77096',), ('77445',), ('77401',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77096',), ('77074',), ('77036',), ('77007',), ('77043',), ('77445',), ('77479',), ('77521',), ('77074',), ('77401',), ('77401',), ('77401',), ('77401',), ('77401',), ('77401',), ('7